In [1]:
import sqlite3
import pandas as pd

# Create/Open the ebookstore database
db = sqlite3.connect('ebookstore') 
cursor = db.cursor()

# Create the books table if it doesn't exist already
cursor.execute('''
          CREATE TABLE IF NOT EXISTS books([id] int, [Title] varchar(255), [Author] varchar(255), [Qty] int)
          ''')

# Commit it to the database
db.commit()

In [2]:
# Define records to tadd to the table
records = [(3001, 'A Tale of Two Cities', 'Charles Dickens', 30),
        (3002, 'Harry Potter and the Philosophers Stone', 'J.K. Rowling', '40'),
        (3003, 'The Lion, the Witch and the Wardrobe', 'C.S. Lewis', 25),
        (3004, 'The Lord of the Rings', 'J.R.R. Tolkien', 37),
        (3005, 'Alice in Wonderland', 'Lewis Carroll', 12)]

In [3]:
# Add records to the books table
cursor.executemany('''
        INSERT INTO books VALUES(?,?,?,?);
          ''', records)

In [5]:
# Define a function to create a current version of the books table as a dataframe
def get_books_df():
    cursor.execute(''' SELECT * from books''')
    global current_records_df
    current_records_df = pd.DataFrame(cursor.fetchall(),columns=['id','title','author','qty'])

In [6]:
# Define a function to add new books to the table
def add_book(book_id, title, author, qty):
    cursor.execute('''
        INSERT INTO books
        VALUES(?, ?, ?, ?)
        ''', (book_id, title, author, qty))
    db.commit()

In [7]:
# Define a function to update books in the table
def update_book(book_id, column_name, new_value):
    cursor.execute(f'''
        UPDATE books SET {column_name} = ? WHERE id = ?
        ''', (new_value, book_id))
    db.commit()

In [8]:
# Define a function to delete books from the table
def delete_book(book_id):
    cursor.execute(f'''
        DELETE FROM books WHERE id = {book_id}
        ''')
    db.commit()

In [9]:
# Define a function to get specific book details
def get_book(book_id):
    cursor.execute(f'''
    SELECT * FROM books WHERE id = {book_id}''')
    return cursor.fetchall()

In [10]:
# Define a function to check if value exists in a column
def check_value(column_name, value):
    if value in current_records_df[column_name].unique():
        return True
    else:
        return False

In [11]:
while True: 
    # Get the current content as a datafram
    get_books_df()
    # Present the menu
    action = input('''
    Please select one of the following options:

    1. Enter Book (e)
    2. Update Book (u)
    3. Delete Book (d)
    4. Search Book (s)
    5. Exit (x)
    ''')
     
    if action == 'e':
        
        # If users selects to enter a book, ask for all the details of the book
        while True:
            book_id = input("Please enter the book id: ")
            title = input("Please enter the title of the book: ")
            author = input("Please enter the name of the author: ")
            qty = input("Please enter the quantity in store: ")
            
            # Ensure book id and qty are convertable to int
            try:
                book_id = int(book_id)
                qty = int(qty)
                break
            except:
                print("Please ensure both book id and quantity are in integer format")
        
        # Add the book to the table and inform the user
        add_book(book_id, title, author, qty)
        # Update the current books dataframe
        get_books_df()
        print("Book successfully added")

    elif action == 'u':
        
        # If users selects to update a book, ask for book id and check if valid
        while True:
            try:
                book_id = int(input("Please enter the book id: "))
                if check_value('id',book_id)==False:
                    print("Please enter a valid book id")
                else:
                    break
            except:
                print("Please ensure book id is in integer format")
        
        # Ask for the column to update and check if valid. Save reponse in correct data type.
        while True:
            column_name = input("Please enter the name of the column to update (id, title, author, qty): ").lower()
            if column_name in ['id','qty']:
                new_value = int(input(f"Please enter the new {column_name} for book {book_id}"))
                break
            elif column_name in ['title', 'author']:
                new_value = input(f"Please enter the new {column_name} for book {book_id}")
                break
            else:
                print("This is not a correct column name")
        
        # Update the book and inform the user
        update_book(book_id, column_name, new_value)
        # Update the current books dataframe
        get_books_df()
        print("Book succassfully updated")

    elif action == 'd':
        
        # If user selects to delete a book, ask for book id and check if valid
        while True:  
            try:
                book_id = int(input("Please enter the book id to delete: "))
                if check_value('id',book_id)==False:
                    print("Please enter a valid book id")
                else:
                    break
            except:
                print("Please ensure book id is in integer format")
                
        # Delete the book and inform the user
        delete_book(book_id)
        # Update the current books dataframe
        get_books_df()
        print("Book successfully deleted")

    elif action == 's':
        
        # If user selects to search for a book, ask for book id and check if valid
        while True:
            try:
                book_id = int(input("Please enter the book id: "))
                if check_value('id',book_id)==False:
                    print("Please enter a valid book id")
                else:
                    break
            except:
                print("Please ensure book id is in integer format")
        
        # Print the results of the search
        print("Here are the books the match your criteria:\n")
        print(get_book(book_id))
        
    elif action == 'x':
        
        # If user selects to exit, end the program
        print("Goodbye!")
        db.close()
        break
    
    # If user does not select one of the available options, display an error message and go back to the menu
    else:
        print('\nOops! It looks like we do not support this action.')


    Please select one of the following options:

    1. Enter Book (e)
    2. Update Book (u)
    3. Delete Book (d)
    4. Search Book (s)
    5. Exit (x)
    e
Please enter the book id: 34
Please enter the title of the book: toto
Please enter the name of the author: tototo
Please enter the quantity in store: 45
Book successfully added

    Please select one of the following options:

    1. Enter Book (e)
    2. Update Book (u)
    3. Delete Book (d)
    4. Search Book (s)
    5. Exit (x)
    s
Please enter the book id: 34
Here are the books the match your criteria:

[(34, 'toto', 'tototo', 45)]

    Please select one of the following options:

    1. Enter Book (e)
    2. Update Book (u)
    3. Delete Book (d)
    4. Search Book (s)
    5. Exit (x)
    u
Please enter the book id: 34
Please enter the name of the column to update (id, title, author, qty): title
Please enter the new title for book 34gogo
Book succassfully updated

    Please select one of the following options:

    1